# データの結合

本節では `Series` および `DataFrame` を結合する方法について学びます。

データを結合する方法は次の2つに分類されます。

- 縦に連結
- 横に連結

データを横に結合する場合、次のいずれかをキーとして結合します。

- インデックス(行のラベル)
- 特定の列

## Seriesの結合

In [1]:
import pandas as pd

s1 = pd.Series([1, 2, 3], index=list("abc"))
s2 = pd.Series([10, 20, 30], index=list("acd"))

Seriesを縦に結合する場合は `concat` 関数を実行します。 `concat` 関数には結合対象となるオブジェクトが格納されたリストを渡します。

In [2]:
pd.concat([s1, s2])

a     1
b     2
c     3
a    10
c    20
d    30
dtype: int64

引数 `ignore_index` に `True` を渡すと、インデックスを振り直します。

In [3]:
pd.concat([s1, s2], ignore_index=True)

0     1
1     2
2     3
3    10
4    20
5    30
dtype: int64

`DataFrame` のコンストラクタから生成する場合は、複数の `Series` をリストなどに格納して渡します。

In [4]:
pd.DataFrame([s1, s2])

,a,b,c,d
0,1.0,2.0,3.0,NaN
1,10.0,NaN,20.0,30.0


In [5]:
pd.DataFrame([s1, s2]).T

,0,1
a,1.0,10.0
b,2.0,NaN
c,3.0,20.0
d,NaN,30.0


## SeriesとDataFrameの結合

In [6]:
df1 = pd.DataFrame(
    [[1, 10, 100], [2, 20, 200], [3, 30, 300]], index=list("abc"), columns=list("ABC")
)

In [7]:
df1

,A,B,C
a,1,10,100
b,2,20,200
c,3,30,300


`DafaFrame` に `Series` を縦に結合するには `concat` 関数を実行します。

In [8]:
pd.concat([df1, s1])

,A,B,C,0
a,1.0,10.0,100.0,NaN
b,2.0,20.0,200.0,NaN
c,3.0,30.0,300.0,NaN
a,NaN,NaN,NaN,1.0
b,NaN,NaN,NaN,2.0
c,NaN,NaN,NaN,3.0


`DafaFrame` に `Series` を横に結合するには `concat` 関数の引数 `axis` に `1` を渡して実行します。

In [9]:
pd.concat([df1, s1], axis=1)

,A,B,C,0
a,1,10,100,1
b,2,20,200,2
c,3,30,300,3


`DataFrame` の列に代入する方法もあります。この場合は元の `DataFrame` を破壊的に書き換えます。

In [10]:
df1_copy = df1.copy()
df1_copy["S1"] = s1
df1_copy

,A,B,C,S1
a,1,10,100,1
b,2,20,200,2
c,3,30,300,3


In [11]:
df1_copy["S2"] = s2
df1_copy

,A,B,C,S1,S2
a,1,10,100,1,10.0
b,2,20,200,2,NaN
c,3,30,300,3,20.0


## DataFrameとDataFrameの結合

In [12]:
df2 = pd.DataFrame(
    [[4, 10, 100], [2, 20, 200], [3, 30, 400]], index=list("acd"), columns=list("ACD")
)

In [13]:
df1

,A,B,C
a,1,10,100
b,2,20,200
c,3,30,300


In [14]:
df2

,A,C,D
a,4,10,100
c,2,20,200
d,3,30,400


`DataFrame` どうしを縦に結合する場合は `concat` 関数を実行します。

In [15]:
pd.concat([df1, df2])

,A,B,C,D
a,1,10.0,100,NaN
b,2,20.0,200,NaN
c,3,30.0,300,NaN
a,4,NaN,10,100.0
c,2,NaN,20,200.0
d,3,NaN,30,400.0


引数 `ignore_index` に `True` を渡すと、インデックスを振り直します。

In [16]:
pd.concat([df1, df2], ignore_index=True)

,A,B,C,D
0,1,10.0,100,NaN
1,2,20.0,200,NaN
2,3,30.0,300,NaN
3,4,NaN,10,100.0
4,2,NaN,20,200.0
5,3,NaN,30,400.0


`concat` 関数を実行して `DafaFrame` を横に結合する場合は、引数 `axis` に `1` を渡します。

この場合、 **インデックスがキー** になります 。

In [17]:
pd.concat([df1, df2], axis=1)

,A,B,C,A,C,D
a,1.0,10.0,100.0,4.0,10.0,100.0
b,2.0,20.0,200.0,NaN,NaN,NaN
c,3.0,30.0,300.0,2.0,20.0,200.0
d,NaN,NaN,NaN,3.0,30.0,400.0


**特定の列をキー** として、 `DataFrame` を横に結合する場合には `merge` メソッドを実行します。引数 `on` にキーとなる列名を指定します。

In [18]:
pd.merge(df1, df2, on="A")

,A,B,C_x,C_y,D
0,2,20,200,20,200
1,3,30,300,30,400


キーとなる列名が異なる場合には、引数 `left_on` および引数 `right_on` にそれぞれ列名を指定します。

In [19]:
pd.merge(df1, df2, left_on="C", right_on="D")

,A_x,B,C_x,A_y,C_y,D
0,1,10,100,4,10,100
1,2,20,200,2,20,200


`merge` メソッドの結合方法はデフォルトで内部結合(inner)です。結合方法は引数 `how` で指定します。

- left: 左外部結合
- right: 右外部結合
- outer: 外部結合

In [20]:
pd.merge(df1, df2, on="A", how="left")

,A,B,C_x,C_y,D
0,1,10,100,NaN,NaN
1,2,20,200,20.0,200.0
2,3,30,300,30.0,400.0


In [21]:
pd.merge(df1, df2, on="A", how="right")

,A,B,C_x,C_y,D
0,4,NaN,NaN,10,100
1,2,20.0,200.0,20,200
2,3,30.0,300.0,30,400


In [22]:
pd.merge(df1, df2, on="A", how="outer")

,A,B,C_x,C_y,D
0,1,10.0,100.0,NaN,NaN
1,2,20.0,200.0,20.0,200.0
2,3,30.0,300.0,30.0,400.0
3,4,NaN,NaN,10.0,100.0
